In [28]:
#Librerías

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Cargar el dataset (usar el mismo del TP1)
data = pd.read_csv("bank.csv")

# Mostrar las primeras filas
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


In [29]:
#Preprocesamiento de los datos

# Columnas con texto (categóricas)
columnas_categoricas = ['job', 'marital', 'education', 'default', 'housing',
                        'loan', 'contact', 'month', 'poutcome']

# Columnas con números
columnas_numericas = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

# Variables de entrada (X) y salida (y)
X = data[columnas_categoricas + columnas_numericas]
y = data['deposit'].map({'yes': 1, 'no': 0})  # convertir a 0 y 1

# Transformar texto a números (One Hot Encoding) y escalar los valores
transformador = ColumnTransformer([
    ('categoricas', OneHotEncoder(handle_unknown='ignore'), columnas_categoricas),
    ('numericas', StandardScaler(), columnas_numericas)
])

X = transformador.fit_transform(X)

print("datos procesados:", X.shape)

datos procesados: (11162, 51)


In [30]:
#Separar los datos

# 60% entrenamiento, 20% validación, 20% test
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=0)

print("entrenamiento:", X_train.shape)
print("validación:", X_val.shape)
print("test:", X_test.shape)

entrenamiento: (6696, 51)
validación: (2233, 51)
test: (2233, 51)


In [31]:
#Primera red neuronal (modelo A)

modeloA = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

modeloA.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar
histA = modeloA.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), verbose=0)

# Evaluar
val_lossA, val_accA = modeloA.evaluate(X_val, y_val, verbose=0)
print("modelo A - accuracy validación:", val_accA)

C:\Users\Famil\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


modelo A - accuracy validación: 0.8387818932533264


In [32]:
#Segunda red neuronal (modelo B)

modeloB = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

modeloB.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar
histB = modeloB.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), verbose=0)

# Evaluar
val_lossB, val_accB = modeloB.evaluate(X_val, y_val, verbose=0)
print("modelo B - accuracy validación:", val_accB)

modelo B - accuracy validación: 0.8213165998458862


In [33]:
#Tercera red neuronal (modelo C)

modeloC = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

modeloC.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar
histC = modeloC.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), verbose=0)

# Evaluar
val_lossC, val_accC = modeloC.evaluate(X_val, y_val, verbose=0)
print("modelo C - accuracy validación:", val_accC)

modelo C - accuracy validación: 0.8428123593330383


In [34]:
#Elegir el mejor modelo y probarlo en el test

# Ver cuál tuvo mejor accuracy
mejor_modelo = None
mejor_acc = 0

if val_accA > mejor_acc:
    mejor_acc = val_accA
    mejor_modelo = modeloA
if val_accB > mejor_acc:
    mejor_acc = val_accB
    mejor_modelo = modeloB
if val_accC > mejor_acc:
    mejor_acc = val_accC
    mejor_modelo = modeloC

print("mejor modelo elegido con accuracy:", mejor_acc)

# Entrenar ese modelo con todo el entrenamiento (train + val)
mejor_modelo.fit(X_train_full, y_train_full, epochs=20, verbose=0)

# Evaluar en test
test_loss, test_acc = mejor_modelo.evaluate(X_test, y_test, verbose=0)
print("accuracy final en test:", test_acc)

mejor modelo elegido con accuracy: 0.8428123593330383
accuracy final en test: 0.8392297625541687
